In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
import re
import datetime


path = '/content/drive/MyDrive/Aviator/Data1/bd_pseudo_1.csv'
#path = '/run/user/1000/gvfs/google-drive:host=gmail.com,user=ozieljr14/0AAGFG0sJpjdeUk9PVA/1BQG9CYJDzM17YcKGZ2Bv-3yo-ijSYBkO/1BZWZDK2-MdUUKwgiqxS8ZDuDxjsGlLzb/bd_pseudo_1.csv'

data = pd.read_csv(path)

t = 0
print(len(data))
t0 = int(input("Maximo: "))
tamanho_data = len(data)

odd = []
horario = []
odd_ajustada = []

while t <= t0:
    if t < tamanho_data:
        value = data['odd'][t]
        if isinstance(value, float):
            odd.append(np.nan)  # or any other value to represent missing data
        else:
            aux = value.split('\n\n')
            a1 = aux[0].replace(')I', '')
            a2 = a1.replace(')', '')
            a3 = a2.replace('|', '1')
            a4 = a3.replace('?', '7')
            a5 = re.sub('[^0-9.]', '', a4)
            odd.append(float(a5))
        horario.append(data['hora_criacao'][t])

    t += 1

for name in odd:
    ajuste = float(name - 1.25)
    odd_ajustada.append(ajuste)

horas = []
minutos = []
segundos = []
for item in horario:
    dt = datetime.datetime.strptime(item, "%a %b %d %H:%M:%S %Y")
    horas.append(dt.hour)
    minutos.append(dt.minute)
    segundos.append(dt.second)

data1 = pd.DataFrame({'Odd_ajustada': odd, 'hora': horas, 'minutos': minutos, 'segundos': segundos, 'horario': horario})

data2 = pd.DataFrame({'horario': horario, 'Odd_ajustada': odd})

tempo = []
for i in range(0, len(data2)-1):
    #print(data2['horario'][i])
    name = pd.to_datetime(data2['horario'][i + 1]) - pd.to_datetime(data2['horario'][i])
    tempo.append(name)


# Convertendo para segundos
tempo_em_segundos = [t.total_seconds() for t in tempo]
tempo_em_segundos.append(12)
# Adicionando a coluna de tempo em segundos ao DataFrame
data2['tempo_em_segundos'] = tempo_em_segundos

dados_filtrados = data2[data2['Odd_ajustada'] < 20]

###Data final
#data = dados_filtrados
data = data1

# Verificar se existem valores NaN na coluna "Odd_ajustada"
has_nan = data['Odd_ajustada'].isna().any()
if has_nan:
    print("Existem valores NaN na coluna 'Odd_ajustada'.")
else:
    print("Não existem valores NaN na coluna 'Odd_ajustada'.")

26401
Maximo: 26401
Não existem valores NaN na coluna 'Odd_ajustada'.


In [2]:
data.head(20)

,Odd_ajustada,hora,minutos,segundos,horario
0,13.89,0,28,6,Sat May 27 00:28:06 2023
1,1.44,0,28,18,Sat May 27 00:28:18 2023
2,3.39,0,28,42,Sat May 27 00:28:42 2023
3,1.27,0,28,56,Sat May 27 00:28:56 2023
4,5.16,0,29,26,Sat May 27 00:29:26 2023
5,2.65,0,29,50,Sat May 27 00:29:50 2023
6,1.00,0,30,3,Sat May 27 00:30:03 2023
7,1.81,0,30,18,Sat May 27 00:30:18 2023
8,1.88,0,30,38,Sat May 27 00:30:38 2023
9,1.60,0,30,54,Sat May 27 00:30:54 2023


In [5]:
data['Odd_ajustada'].describe()

count    26401.000000
mean        13.764437
std        502.107957
min          1.000000
25%          1.290000
50%          1.920000
75%          3.830000
max      70126.000000
Name: Odd_ajustada, dtype: float64

In [ ]:
# Extrair as features (horário) e o target (Odd_ajustada)
features = data["horario"].values
target = data["Odd_ajustada"].values

# Pré-processamento dos dados
scaler = MinMaxScaler()
target_scaled = scaler.fit_transform(target.reshape(-1, 1))

# Tokenizar os horários
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(features)

# Construir o modelo da rede neural
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 32, input_length=1),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1)
])

# Compilar o modelo
model.compile(optimizer="adam", loss="mean_squared_error")

# Definir o número de folds para a validação cruzada
n_splits = 5

# Criar os objetos para realizar a validação cruzada
kf = KFold(n_splits=n_splits)

# Listas para armazenar as métricas de cada fold
mae_scores = []
mse_scores = []
r2_scores = []
rmse_scores = []
mape_scores = []

# Realizar a validação cruzada
for train_index, test_index in kf.split(features):
    # Dividir os dados em conjunto de treinamento e teste para o fold atual
    fold_features_train = features[train_index]
    fold_target_train = target_scaled[train_index]
    fold_features_test = features[test_index]
    fold_target_test = target_scaled[test_index]

    # Tokenizar os horários do fold atual
    fold_features_train_encoded = tokenizer.texts_to_sequences(fold_features_train)
    fold_features_test_encoded = tokenizer.texts_to_sequences(fold_features_test)

    # Pad dos sequências do fold atual
    max_sequence_length = max(len(seq) for seq in fold_features_train_encoded + fold_features_test_encoded)
    fold_features_train_padded = tf.keras.preprocessing.sequence.pad_sequences(
        fold_features_train_encoded, maxlen=max_sequence_length
    )
    fold_features_test_padded = tf.keras.preprocessing.sequence.pad_sequences(
        fold_features_test_encoded, maxlen=max_sequence_length
    )

    # Treinar o modelo no fold atual
    model.fit(fold_features_train_padded, fold_target_train, epochs=25, batch_size=5)

    # Avaliar o modelo no fold de teste
    loss = model.evaluate(fold_features_test_padded, fold_target_test)

    # Fazer previsões no fold de teste
    predictions = model.predict(fold_features_test_padded)

    # Inverter a escala das previsões
    predictions_scaled = scaler.inverse_transform(predictions)

    # Calcular o erro médio absoluto (MAE) e o erro médio quadrático (MSE)
    fold_mae = mean_absolute_error(fold_target_test, predictions_scaled)
    fold_mse = mean_squared_error(fold_target_test, predictions_scaled)

    # Calcular o coeficiente de determinação R-squared (R2)
    fold_r2 = r2_score(fold_target_test, predictions_scaled)

    # Calcular a raiz quadrada do erro médio quadrático (RMSE)
    fold_rmse = np.sqrt(fold_mse)

    # Calcular o erro médio percentual absoluto (MAPE)
    fold_mape = mean_absolute_percentage_error(fold_target_test, predictions_scaled)

    # Armazenar as métricas do fold atual
    mae_scores.append(fold_mae)
    mse_scores.append(fold_mse)
    r2_scores.append(fold_r2)
    rmse_scores.append(fold_rmse)
    mape_scores.append(fold_mape)


# Calcular as médias das métricas de todos os folds
cv_mae = np.mean(mae_scores)
cv_mse = np.mean(mse_scores)
cv_r2 = np.mean(r2_scores)
cv_rmse = np.mean(rmse_scores)
cv_mape = np.mean(mape_scores)

# Imprimir as métricas de validação cruzada
print("Erro médio absoluto (MAE) usando validação cruzada:", cv_mae)
print("Erro médio quadrático (MSE) usando validação cruzada:", cv_mse)
print("R-squared (R2) usando validação cruzada:", cv_r2)
print("Root Mean Squared Error (RMSE) usando validação cruzada:", cv_rmse)
print("Mean Absolute Percentage Error (MAPE) usando validação cruzada:", cv_mape)


Epoch 1/25
4224/4224 [==============================] - 22s 5ms/step - loss: 6.5591e-05
Epoch 2/25
4224/4224 [==============================] - 20s 5ms/step - loss: 6.4652e-05
Epoch 3/25
4224/4224 [==============================] - 19s 4ms/step - loss: 6.4317e-05
Epoch 4/25
4224/4224 [==============================] - 19s 4ms/step - loss: 6.4487e-05
Epoch 5/25
4224/4224 [==============================] - 29s 7ms/step - loss: 6.4081e-05
Epoch 6/25
4224/4224 [==============================] - 21s 5ms/step - loss: 6.4070e-05
Epoch 7/25
4224/4224 [==============================] - 19s 4ms/step - loss: 6.4079e-05
Epoch 8/25
4224/4224 [==============================] - 19s 4ms/step - loss: 6.4028e-05
Epoch 9/25
4224/4224 [==============================] - 20s 5ms/step - loss: 6.3840e-05
Epoch 10/25
4224/4224 [==============================] - 19s 5ms/step - loss: 6.4096e-05
Epoch 11/25
4224/4224 [==============================] - 18s 4ms/step - loss: 6.3970e-05
Epoch 12/25
4224/4224 [=======

In [ ]:
# Calcular o tempo médio de cada odd
odd_times = pd.to_datetime(data["horario"])
time_diff = odd_times - odd_times.shift(1)
average_time = time_diff.mean()

# Criar o array de novos horários com base no tempo médio
new_horarios = []
last_horario = odd_times.iloc[-1]
new_horario = last_horario + average_time
new_horarios.append(new_horario)
i = 0
while i < 10:
    new_horario = new_horarios[i] + average_time
    new_horarios.append(new_horario)
    i += 1

# Converter os novos horários para o formato de string
new_features = [horario.strftime("%I:%M:%S %p") for horario in new_horarios]

# Tokenizar os novos horários
new_features_encoded = tokenizer.texts_to_sequences(new_features)
new_features_padded = tf.keras.preprocessing.sequence.pad_sequences(new_features_encoded, maxlen=max_sequence_length)

# Fazer previsões com os novos dados
new_predictions = model.predict(new_features_padded)

# Inverter a escala das previsões
new_predictions_scaled = scaler.inverse_transform(new_predictions)

# Obter a última entrada da coluna "Odd_ajustada"
last_odd = data['Odd_ajustada'].iloc[-1]
print("Última entrada da coluna 'Odd_ajustada':", last_odd)

# Imprimir as previsões
for i, prediction in enumerate(new_predictions_scaled):
    print(f"Previsão {i + 1}: {prediction[0]} - Horário: {new_features[i]}")


1/1 [==============================] - 0s 28ms/step
Última entrada da coluna 'Odd_ajustada': 1.55
Previsão 1: 6.291195869445801 - Horário: 01:07:27 AM
Previsão 2: 10.435081481933594 - Horário: 01:11:56 AM
Previsão 3: 4.072478294372559 - Horário: 01:16:26 AM
Previsão 4: 2.471709728240967 - Horário: 01:20:55 AM
Previsão 5: 8.074700355529785 - Horário: 01:25:24 AM
Previsão 6: 14.39137077331543 - Horário: 01:29:54 AM
Previsão 7: 8.020745277404785 - Horário: 01:34:23 AM
Previsão 8: 30.6621150970459 - Horário: 01:38:52 AM
Previsão 9: 18.440750122070312 - Horário: 01:43:22 AM
Previsão 10: 14.749234199523926 - Horário: 01:47:51 AM
Previsão 11: -0.9540907740592957 - Horário: 01:52:20 AM


In [ ]:
# Converter a coluna 'horario' para o formato de data e hora
data['horario'] = pd.to_datetime(data['horario'])

# Defina a coluna 'horario' como o índice do DataFrame
data.set_index('horario', inplace=True)

# Reamostrar os dados para um intervalo de tempo fixo (por exemplo, a cada 5 minutos)
data_resampled = data.resample('5T').mean()

# Calcular o intervalo médio entre as amostras de odds em segundos
intervalo_medio = data_resampled['tempo_em_segundos'].mean()

# Criar um modelo ARIMA
model_arima = ARIMA(data_resampled['tempo_em_segundos'], order=(1, 0, 0))  # Pode ajustar a ordem do ARIMA conforme necessário
model_fit = model_arima.fit()

i = 0
while i < 15:  # Vamos fazer 15 previsões

    # Fazer previsões do próximo intervalo de tempo com base no modelo ARIMA
    proximo_intervalo = model_fit.forecast(steps=1)[0]

    # Calcular o próximo horário previsto
    ultimo_horario = data_resampled.index[-1]
    proximo_horario_previsto = ultimo_horario + pd.Timedelta(seconds=intervalo_medio)

    # Adicionar o próximo horário previsto ao DataFrame
    data_resampled.loc[proximo_horario_previsto] = proximo_intervalo

    # Atualizar o intervalo médio com o valor mais recente
    intervalo_medio = data_resampled['tempo_em_segundos'].mean()

    i += 1

# Calcular os próximos horários previstos
proximos_horarios_previstos = data_resampled.index[-15:]

#print("Próximos horários previstos:")
#for horario in proximos_horarios_previstos:
#    print(horario)

# Converter os novos horários para o formato de string
new_features = [horario.strftime("%I:%M:%S %p") for horario in proximos_horarios_previstos]

# Tokenizar os novos horários
new_features_encoded = tokenizer.texts_to_sequences(new_features)
new_features_padded = tf.keras.preprocessing.sequence.pad_sequences(new_features_encoded, maxlen=max_sequence_length)

# Fazer previsões com os novos dados
new_predictions = model.predict(new_features_padded)

# Inverter a escala das previsões
new_predictions_scaled = scaler.inverse_transform(new_predictions)

# Obter a última entrada da coluna "Odd_ajustada"
last_odd = data['Odd_ajustada'].iloc[-1]
print("Última entrada da coluna 'Odd_ajustada':", last_odd)

# Imprimir as previsões
for i, prediction in enumerate(new_predictions_scaled):
    print(f"Previsão {i + 1}: {prediction[0]} - Horário: {new_features[i]}")

<ipython-input-25-42e566691df6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['horario'] = pd.to_datetime(data['horario'])


Próximos horários previstos:
2023-07-19 01:41:58.261233621
2023-07-19 01:48:56.517609539
2023-07-19 01:55:54.769134563
2023-07-19 02:02:53.015815487
2023-07-19 02:09:51.257659090
2023-07-19 02:16:49.494672138
2023-07-19 02:23:47.726861382
2023-07-19 02:30:45.954233560
2023-07-19 02:37:44.176795394
2023-07-19 02:44:42.394553594
2023-07-19 02:51:40.607514854
2023-07-19 02:58:38.815685855
2023-07-19 03:05:37.019073265
2023-07-19 03:12:35.217683736
2023-07-19 03:19:33.411523908
1/1 [==============================] - 0s 37ms/step
Última entrada da coluna 'Odd_ajustada': 8.06
Previsão 1: 3.825758934020996 - Horário: 01:41:58 AM
Previsão 2: 7.317008972167969 - Horário: 01:48:56 AM
Previsão 3: 16.76764488220215 - Horário: 01:55:54 AM
Previsão 4: -2.1285030841827393 - Horário: 02:02:53 AM
Previsão 5: 4.598487854003906 - Horário: 02:09:51 AM
Previsão 6: 16.421428680419922 - Horário: 02:16:49 AM
Previsão 7: 23.975318908691406 - Horário: 02:23:47 AM
Previsão 8: -0.7105675339698792 - Horário: 02:30

In [ ]:
# Fazer previsões com novos dados
new_features = ["06:55:05 AM", "6:55:10 AM", "6:55:15 AM", "6:55:20 AM", "06:55:25 AM","06:55:30 AM","06:55:35 AM","06:55:40 AM"]  # Exemplo de novos horários

# Tokenizar os novos horários
new_features_encoded = tokenizer.texts_to_sequences(new_features)
new_features_padded = tf.keras.preprocessing.sequence.pad_sequences(new_features_encoded, maxlen=max_sequence_length)

# Fazer previsões com os novos dados
new_predictions = model.predict(new_features_padded)

# Inverter a escala das previsões
new_predictions_scaled = scaler.inverse_transform(new_predictions)

# Imprimir as previsões
for i, prediction in enumerate(new_predictions_scaled):
    print(f"Previsão {i + 1}: {prediction[0]}")

1/1 [==============================] - 0s 496ms/step
Previsão 1: 4.38295316696167
Previsão 2: -2.900407314300537
Previsão 3: 0.28372958302497864
Previsão 4: 8.288626670837402
Previsão 5: -0.3044455945491791
Previsão 6: 5.559839725494385
Previsão 7: 0.9186298251152039
Previsão 8: -2.527514696121216


In [ ]:
# Obter a última entrada da coluna "Odd_ajustada"
last_odd = data['Odd_ajustada'].iloc[-1]
print("Última entrada da coluna 'Odd_ajustada':", last_odd)
print(predictions_scaled[:25])


Última entrada da coluna 'Odd_ajustada': 2.28
[[7.8037667]
 [4.772703 ]
 [4.2964296]
 [3.6401193]
 [4.227021 ]
 [3.9743814]
 [4.684922 ]
 [5.4031963]
 [4.7539263]
 [3.6704369]
 [3.6080909]
 [7.0796924]
 [8.545972 ]
 [9.636379 ]
 [7.8958006]
 [8.836828 ]
 [5.6680665]
 [5.502526 ]
 [8.349851 ]
 [4.163399 ]
 [5.0716453]
 [5.296194 ]
 [6.5764794]
 [7.7780547]
 [4.097716 ]]


In [ ]:

# Calcular o tempo médio de cada odd
odd_times = pd.to_datetime(data["horario"])
time_diff = odd_times - odd_times.shift(1)
average_time = time_diff.mean()

# Criar o array de novos horários com base no tempo médio
new_horarios = []
last_horario = odd_times.iloc[-1]
new_horario = last_horario + average_time
new_horarios.append(new_horario)
i = 0
while i < 10:
  new_horario = new_horarios[i] + average_time
  new_horarios.append(new_horario)

  i += 1

# Imprimir o tempo médio da odd
print(f"Tempo médio da odd: {average_time}")

# Imprimir os novos horários
for i, new_horario in enumerate(new_horarios):
    print(f"Novo horário {i + 1}: {new_horario}")


Tempo médio da odd: 0 days 00:02:44.864398298
Novo horário 1: 2023-06-11 06:36:16.864398298
Novo horário 2: 2023-06-11 06:39:01.728796596
Novo horário 3: 2023-06-11 06:41:46.593194894
Novo horário 4: 2023-06-11 06:44:31.457593192
Novo horário 5: 2023-06-11 06:47:16.321991490
Novo horário 6: 2023-06-11 06:50:01.186389788
Novo horário 7: 2023-06-11 06:52:46.050788086
Novo horário 8: 2023-06-11 06:55:30.915186384
Novo horário 9: 2023-06-11 06:58:15.779584682
Novo horário 10: 2023-06-11 07:01:00.643982980
Novo horário 11: 2023-06-11 07:03:45.508381278
